# Item Level Explainability - Amazon Forecast 

Our goal is to train a forecasting model with Amazon Forecast and explain the resultant model in order to understand how different features are impacting the predictions using Forecast Explainability.

Explainability helps you better understand how the attributes in your datasets impact your forecasts. Amazon Forecast uses a metric called Impact scores to quantify the relative impact of each attribute and determine whether they increase or decrease forecast values.

To enable Forecast Explainability, your predictor must include at least one of the following: related time series, item metadata, or additional datasets like Holidays and the Weather Index.

CreateExplainability accepts either a Predictor ARN or Forecast ARN. To receive aggregated Impact scores for all time series and time points in your datasets, provide a Predictor ARN. To receive Impact scores for specific time series and time points, provide a Forecast ARN.


To do this, we will predict the order quantity for 20 musical instruments for US stores belonging to MyMusicCompany Inc, with monthly frequency for a 12 month forecast horizon. Time-series forecasting is important to avoid the costs related to under and over forecasting, in this case specifically for order quantities for different musical instruments. The data includes dates, instrument models and order quantities. The data contains related time-varying features including Loss Rate which represents items that get damaged during transportation, and Customer Request, which represents the number of customers on the wait list for an item. The data contains one static feature, Model Type, which represents the category the Model Id belongs to. We will train our model with the built-in holidays data provided by Amazon Forecast. We will then examine how the features in the data impact the order quantity using Explainability. 

Note that the impact scores, including those shown in this notebook, may differ between jobs due to some inherent randonmess in how impact scores are computed.


<br>

Note: the data used in this notebook is a synthetic dataset generated for the purposes of educating you on how to use the feature.

**This notebook covers generating explainability for forecasting models through Amazon Forecast.** 
<li><a href="https://aws.amazon.com/blogs/machine-learning/understand-drivers-that-influence-your-forecasts-with-explainability-impact-scores-in-amazon-forecast/" target="_blank">See blog announcement understand drivers that influence your
forecasts with explainability impact scores in Amazon
Forecast.</a></li>

<br>


# Table of Contents

* Step 0: [Setting up](#setup)
* Step 1: [Importing the Data into Forecast](#import)
 * Step 1a: [Creating a Dataset Group](#createDSG)
 * Step 1b: [Creating a Target Dataset](#targetDS)
 * Step 1c: [Creating an RTS Dataset](#RTSDS)
 * Step 1d: [Creating an IM Dataset](#IMDS)
 * Step 1e: [Update the Dataset Group](#updateDSG)
 * Step 1f: [Creating a Target Time Series Dataset Import Job](#targetImport)
 * Step 1g: [Creating a Related Time Series Dataset Import Job](#RTSImport)
 * Step 1h: [Creating an Item Metadata Import Job](#IMImport)
* Step 2a: [Train an AutoPredictor](#AutoPredictor)
* Step 2b: [Export the model-level explainability](#export)
* Step 2c: [Visualize the model-level explainability](#visualize)
* Step 3: [Create a Forecast](#forecast)
* Step 4a: [Create explainability for specific time-series](#itemLevelExplainability)
* Step 4b: [Create explainability export for specific time-series](#itemLevelExplainabilityExport)
* Step 4c: [Create explainability for specific time-series at time-points](#itemAndTimePointLevelExplainability)
* Step 4d: [Create explainability export for specific time-series at time-points](#itemAndTimePointLevelExplainabilityExport)
* Step 5: [Cleaning up your Resources](#cleanup)


##  Step 0: Setting up

### First let us setup Amazon Forecast<a class="anchor" id="setup">

This section sets up the permissions and relevant endpoints.

In [ ]:
import sys
import os
import shutil
import datetime

import pandas as pd
import numpy as np

# get region from boto3
import boto3
REGION = boto3.Session().region_name

# importing forecast notebook utility from notebooks/common directory
sys.path.insert( 0, os.path.abspath("../../common") )
import util

import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['figure.figsize'] = (15.0, 5.0)

First, let's define a helper function.
This function will make it easier to read in the exported files created as part of an explaiability export into a single pandas dataframe. We'll use this later in the notebook.

In [ ]:
def read_explainability_export(BUCKET_NAME, s3_path):
    """Read explainability export files
       Inputs: 
           BUCKET_NAME = S3 bucket name
           s3_path = S3 path to export files
                         , everything after "s3://BUCKET_NAME/" in S3 URI path to your files
       Return: Pandas dataframe with all files concatenated row-wise
    """
    # set s3 path
    s3 = boto3.resource('s3')
    s3_bucket = boto3.resource('s3').Bucket(BUCKET_NAME)
    s3_depth = s3_path.split("/")
    s3_depth = len(s3_depth) - 1
    
    # set local path
    local_write_path = "explainability_exports"
    if (os.path.exists(local_write_path) and os.path.isdir(local_write_path)):
        shutil.rmtree('explainability_exports')
    if not(os.path.exists(local_write_path) and os.path.isdir(local_write_path)):
        os.makedirs(local_write_path)
    
    # concat part files
    part_filename = ""
    part_files = list(s3_bucket.objects.filter(Prefix=s3_path))
    print(f"Number .part files found: {len(part_files)}")
    for file in part_files:
        # There will be a collection of CSVs, modify this to go get them all
        if "csv" in file.key:
            part_filename = file.key.split('/')[s3_depth]
            window_object = s3.Object(BUCKET_NAME, file.key)
            file_size = window_object.content_length
            if file_size > 0:
                s3.Bucket(BUCKET_NAME).download_file(file.key, local_write_path+"/"+part_filename)
        
    # Read from local dir and combine all the part files
    temp_dfs = []
    for entry in os.listdir(local_write_path):
        if os.path.isfile(os.path.join(local_write_path, entry)):
            df = pd.read_csv(os.path.join(local_write_path, entry), index_col=None, header=0)
            temp_dfs.append(df)

    # Return assembled .part files as pandas Dataframe
    fcst_df = pd.concat(temp_dfs, axis=0, ignore_index=True, sort=False)
    return fcst_df

Configure the S3 bucket name and region name for this lesson.

- If you don't have an S3 bucket, create it first on S3.
- Although we have set the region to us-west-2 as a default value below, you can choose any of the regions that the service is available in.

In [ ]:
bucket_name = input("\nEnter S3 bucket name for uploading the data:")
default_region = REGION
REGION = input(f"region [enter to accept default]: {default_region} ") or default_region 

Connect API session

In [ ]:
session = boto3.Session(region_name=REGION) 
forecast = session.client(service_name='forecast') 

Create the role to provide to Amazon Forecast

In [ ]:
role_name = "ForecastNotebookRole-Explainability"
print(f"Creating Role {role_name} ...")
default_role = util.get_or_create_iam_role( role_name = role_name )
role_arn = default_role

print(f"Success! Created role arn = {role_arn.split('/')[1]}")
print(role_arn)

Verify the steps above were succesful by calling list_predictors()

In [ ]:
forecast.list_predictors()

## Step 1. Importing the Data<a class="anchor" id="import">

In this step, we will create a **Dataset** and **Import** the dataset from S3 to Amazon Forecast. To train a Predictor we will need a **DatasetGroup** that groups the input **Datasets**. So, we will end this step by creating a **DatasetGroup** with the imported **Dataset**.


Define a dataset group name and version number for naming purposes

In [ ]:
project = "explainability_notebook"
idx = 1

## Step 1a. Creating a Dataset Group<a class="anchor" id="createDSG">
First let's create a dataset group and then update it later to add our datasets.

In [ ]:
dataset_group = f"{project}_{idx}"
dataset_arns = []
create_dataset_group_response = forecast.create_dataset_group(
    Domain="CUSTOM",
    DatasetGroupName=dataset_group,
    DatasetArns=dataset_arns)

Below, we specify key input data and forecast parameters.

The forecast frequency for this data is weekly.
The forecast horizon for this data is 12 weeks, which is about 3 months.

In [ ]:
freq = "M"
forecast_horizon = 12
timestamp_format = "yyyy-MM-dd HH:mm:ss"
delimiter = ','

In [ ]:
print(f'Creating dataset group {dataset_group}')

In [ ]:
dataset_group_arn = create_dataset_group_response['DatasetGroupArn']

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 1b. Creating a Target Time Series (TTS) Dataset<a class="anchor" id="targetDS">
In this example, we will define a target time series. This is a required dataset to use the service.

In [ ]:
ts_dataset_name = f"{project}_tts_{idx}"
print(ts_dataset_name)

Next, we specify the schema of our dataset below. Make sure the order of the attributes (columns) matches the raw data in the files. We follow the same three attribute format as the above example.

In [ ]:
ts_schema_val = [
    {"AttributeName": "timestamp", "AttributeType": "timestamp"},
    {"AttributeName": "item_id", "AttributeType": "string"},
    {"AttributeName": "target_value", "AttributeType": "float"}]
ts_schema = {"Attributes": ts_schema_val}

In [ ]:
print(f'Creating target dataset {ts_dataset_name}')

In [ ]:
response = forecast.create_dataset(
    Domain="CUSTOM",
    DatasetType='TARGET_TIME_SERIES',
    DatasetName=ts_dataset_name,
    DataFrequency=freq,
    Schema=ts_schema
  )

In [ ]:
ts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=ts_dataset_arn)

## Step 1c. Creating an Related Time Series (RTS) Dataset<a class="anchor" id="RTSDS">
In this example, we will define a related time series dataset. The columns in the RTS are attributes whose impact can be explained. 

In [ ]:
rts_dataset_name = f"{project}_rts_{idx}"
print(rts_dataset_name)

In [ ]:
rts_schema_val = [
              {"AttributeName": "timestamp", "AttributeType": "timestamp"},
              {"AttributeName": "item_id", "AttributeType": "string"},
              {"AttributeName": "Loss_Rate", "AttributeType": "float"},
              {"AttributeName": "Customer_Request", "AttributeType": "float"}]
rts_schema = {"Attributes": rts_schema_val}

In [ ]:
print(f'Creating RTS dataset {rts_dataset_name}')

In [ ]:
response = forecast.create_dataset(
    Domain="CUSTOM",
    DatasetType='RELATED_TIME_SERIES',
    DataFrequency=freq,
    DatasetName=rts_dataset_name,
    Schema=rts_schema
  )

In [ ]:
rts_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=rts_dataset_arn)

## Step 1d. Creating an Item Metadata (IM) Dataset<a class="anchor" id="IMDS">
In this example, we will define an Item Metadata dataset. This will be a feature whose impact can be explained. 

In [ ]:
im_dataset_name = f"{project}_im_{idx}"
print(im_dataset_name)

In [ ]:
im_schema_val = [
              {"AttributeName": "item_id", "AttributeType": "string"},
              {"AttributeName": "Model_Type", "AttributeType": "string"}]
im_schema = {"Attributes": im_schema_val}

In [ ]:
print(f'Creating IM dataset {im_dataset_name}')

In [ ]:
response = forecast.create_dataset(
    Domain="CUSTOM",
    DatasetType='ITEM_METADATA',
    DatasetName=im_dataset_name,
    Schema=im_schema
  )

In [ ]:
im_dataset_arn = response['DatasetArn']

In [ ]:
forecast.describe_dataset(DatasetArn=im_dataset_arn)

## Step 1e. Updating the dataset group with the datasets we created<a class="anchor" id="updateDSG">
You can have multiple datasets under the same dataset group. Update it with the datasets we created before.

In [ ]:
dataset_arns = []
dataset_arns.append(ts_dataset_arn)
dataset_arns.append(rts_dataset_arn)
dataset_arns.append(im_dataset_arn)

forecast.update_dataset_group(DatasetGroupArn=dataset_group_arn, DatasetArns=dataset_arns)

In [ ]:
forecast.describe_dataset_group(DatasetGroupArn=dataset_group_arn)

## Step 1f. Creating a Target Time Series Dataset Import Job<a class="anchor" id="targetImport">
   
Below, we save the Target Time Series to your bucket on S3, since Amazon Forecast expects to be able to import the data from S3.

In [ ]:
local_file = "instrumentData/TTS.csv"
key = f"{project}/{local_file}"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(local_file)

In [ ]:
ts_s3_data_path = f"s3://{bucket_name}/{project}/{local_file}"
print(ts_s3_data_path)

In [ ]:
ts_dataset_import_job_response = forecast.create_dataset_import_job(
    DatasetImportJobName=dataset_group,
    DatasetArn=ts_dataset_arn,
    DataSource= {
        "S3Config" : {
            "Path": ts_s3_data_path,
            "RoleArn": role_arn
        } 
    },
    TimestampFormat=timestamp_format
    )


In [ ]:
ts_dataset_import_job_arn=ts_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
assert status

## Step 1g. Creating a Related Time Series Dataset Import Job<a class="anchor" id="RTSImport">
Below, we save the Related Time Series to your bucket on S3, since Amazon Forecast expects to be able to import the data from S3.

In [ ]:
local_file = "instrumentData/RTS.csv"
key = f"{project}/{local_file}"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(local_file)

In [ ]:
rts_s3_data_path = f"s3://{bucket_name}/{project}/{local_file}"
print(rts_s3_data_path)

In [ ]:
rts_dataset_import_job_response = forecast.create_dataset_import_job(
    DatasetImportJobName=dataset_group,
    DatasetArn=rts_dataset_arn,
    DataSource= {
        "S3Config" : {
            "Path": rts_s3_data_path,
            "RoleArn": role_arn
        } 
    })

In [ ]:
rts_dataset_import_job_arn=rts_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=rts_dataset_import_job_arn))
assert status

## Step 1h. Creating an Item Metadata Dataset Import Job<a class="anchor" id="IMImport">
Below, we save the Item Metadata to your bucket on S3, since Amazon Forecast expects to be able to import the data from S3.

In [ ]:
local_file = "instrumentData/IM.csv"
key = f"{project}/{local_file}"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(local_file)

In [ ]:
im_s3_data_path = f"s3://{bucket_name}/{project}/{local_file}"
print(im_s3_data_path)

In [ ]:
im_dataset_import_job_response = forecast.create_dataset_import_job(
    DatasetImportJobName=dataset_group,
    DatasetArn=im_dataset_arn,
    DataSource= {
        "S3Config" : {
            "Path": im_s3_data_path,
            "RoleArn": role_arn
        } 
    })

In [ ]:
im_dataset_import_job_arn=im_dataset_import_job_response['DatasetImportJobArn']

In [ ]:
status = util.wait(lambda: forecast.describe_dataset_import_job(DatasetImportJobArn=im_dataset_import_job_arn))
assert status

## Step 2a. Train an AutoPredictor with RTS, IM and Holidays<a class="anchor" id="AutoPredictor">

Next, we will train an AutoPredictor using the dataset group created in step 2 as well as US Holidays

Explainability requires at least one dataset attribute other than the item_id and target_value attributes. So for the predictor we create, impact scores will be generated for RTS columns, IM and Holidays.

You can create Explainability for all forecasts generated from an AutoPredictor. 
In addition to this, at AutoPredictor creation, you have the option to generate model-level explainability. 
We will enable this option for predictor creation by setting:

```python
ExplainPredictor=True
```

In [ ]:
auto_predictor_name = f'holidays_instrument_orders_auto_predictor_{idx}'

print(f'[{auto_predictor_name}] Creating predictor {auto_predictor_name} ...')

In [ ]:
create_predictor_response = forecast.create_auto_predictor(
      PredictorName=auto_predictor_name,
      ForecastHorizon=forecast_horizon,
      ForecastFrequency="M",
      DataConfig=
      {"DatasetGroupArn":dataset_group_arn,
       "AdditionalDatasets":
        [
          {"Name":"holiday",
           "Configuration":
            {"CountryCode":
              ["US"]
            }
          }
        ]
      },
    ExplainPredictor=True
    )

In [ ]:
predictor_arn = create_predictor_response['PredictorArn']

In [ ]:
status = util.wait(lambda: forecast.describe_auto_predictor(PredictorArn=predictor_arn))
assert status

In [ ]:
forecast.describe_auto_predictor(PredictorArn=predictor_arn)

### When we created the AutoPredictor, we also created a model level explainability job
We will wait for the explainability job to be Active, and then we can export it and view the results.

Get the explainability arn from calling describe on the predictor.

In [ ]:
auto_predictor_response = forecast.describe_auto_predictor(PredictorArn=predictor_arn)
explainability_model_level_arn = auto_predictor_response["ExplainabilityInfo"]["ExplainabilityArn"]

In [ ]:
status = util.wait(lambda: forecast.describe_explainability(ExplainabilityArn=explainability_model_level_arn))
assert status

Now that the explainability is Active, we will export the results by creating an explainablity export

## Step 2b. Export the model-level explainability<a class="anchor" id="export">

In [ ]:
explainability_export_name = f"{project}_explainability_export_model_level_{idx}"
explainability_export_destination = f"s3://{bucket_name}/{project}/{explainability_export_name}"

In [ ]:
explainability_export_response = forecast.create_explainability_export(ExplainabilityExportName=explainability_export_name, 
                                                                       ExplainabilityArn=explainability_model_level_arn, 
                                                                       Destination=
                                                                      {"S3Config": 
                                                                        {"Path": explainability_export_destination,
                                                                         "RoleArn": role_arn}
                                                                      }
                                                                      )

In [ ]:
explainability_export_model_level_arn = explainability_export_response['ExplainabilityExportArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_model_level_arn))
assert status

In [ ]:
forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_model_level_arn)

Now, let's load and view the data

In [ ]:
export_data = read_explainability_export(bucket_name, project+"/"+explainability_export_name)

In [ ]:
export_data.style.hide_index()

Impact scores come in two forms: Normalized impact scores and Raw impact scores. Raw impact scores are based on Shapley values and are not scaled or bounded. Normalized impact scores scale the raw scores to a value between -1 and 1 to make comparing scores within the Explainability job easier.

Note that the impact scores, including those shown in this notebook, may differ between jobs due to some inherent randonmess in how impact scores are computed.

Here we can see the aggregatd scores across time-series for features in the model. 

From the scores, Customer_Request has the highest impact driving up the forecasted values, as the normalized impact score is closest to 1.

Loss_Rate has a lower impact then Customer_Request does.

Of the features explained, Holiday_US has the lowest impact on the forecasted values, as the normalized impact score of is closest to 0 (no impact).

It is important to note that Impact scores measure the relative impact of attributes, not the absolute impact. Therefore, Impact scores cannot be used to conclude whether particular attributes improve model accuracy. If an attribute has a low Impact score, that does not necessarily mean that it has a low impact on forecast values; it means that it has a lower impact on forecast values than other attributes used by the predictor. 

## Step 2c. Visualize the model-level explainability<a class="anchor" id="visualize">

![Image View predictor explainability impact scores](images/ModelLevelScores.png "Model level impact scores for the predictor")

We can also view these results on the Amazon Forecast console.

For more details about using the Forecast console to create and view explainabilities, see: https://aws.amazon.com/blogs/machine-learning/understand-drivers-that-influence-your-forecasts-with-explainability-impact-scores-in-amazon-forecast/

## Step 3. Create Forecast <a class="anchor" id="forecast">


In [ ]:
forecast_name = f"{project}_forecast_{idx}"

create_forecast_response = forecast.create_forecast(
    ForecastName=forecast_name,
    PredictorArn = predictor_arn
     )

In [ ]:
forecast_arn = create_forecast_response['ForecastArn']

In [ ]:
status = util.wait(lambda: forecast.describe_forecast(ForecastArn=forecast_arn))
assert status

In [ ]:
forecast.describe_forecast(ForecastArn=forecast_arn)

## Step 4a. Create Explainability for specific time-series <a class="anchor" id="itemLevelExplainability">


We examined the model-level explainability generated during AutoPredictor creation. 

Next we will generate explainability for a set of time-series of our choosing.

To specify a list of time series, upload a CSV file identifying the time series by their item_id and dimension values. You can specify up to 50 time series. You must also define the attributes and attribute types of the time series in a schema.

In this dataset, each time series is only defined by their item_id. 

We will load and view the item subset file stored locally. 

In [ ]:
item_subset_file = "InstrumentData/item_subset.csv"
item_subset_df = pd.read_csv(item_subset_file, names=['item_id'])
item_subset_df.style.hide_index()

Now save the local item susbet file to S3, as Forecast expects to read the file from S3. 

In [ ]:
key = f"{project}/InstrumentData/item_subset.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(item_subset_file)

In [ ]:
item_subset_path = f"s3://{bucket_name}/{key}"
explainability_name = f"{project}_item_level_explainability_{idx}"

To create the explainability using this subset of time-series, configure the following datatypes:

* ExplainabilityConfig - set values for TimeSeriesGranularity to “SPECIFIC” and TimePointGranularity to “ALL”.
```python
ExplainabilityConfig={"TimeSeriesGranularity": "SPECIFIC", "TimePointGranularity": "ALL"}
```
* S3Config - set the values for “Path” to the S3 location of the CSV file and “RoleArn” to a role with access to the S3 bucket.
```python
"S3Config": {"Path": item_subset_path, "RoleArn": role_arn}
```
* Schema - define the “AttributeName” and “AttributeType” for item_id and the dimensions in the time series.
```python
Schema={"Attributes": 
                    [{"AttributeName": "item_id",
                      "AttributeType": "string",
                      "AttributeCategory": "item_id"}
                    ]
                                                               
       }
```

In order to view the explainability results on the console, we set EnableVisualiztion to True.
```python
EnableVisualization=True
```

In [ ]:
create_expainability_response=forecast.create_explainability(ExplainabilityName=explainability_name, 
                                                            ResourceArn=forecast_arn,
                                                            ExplainabilityConfig={"TimeSeriesGranularity": "SPECIFIC", "TimePointGranularity": "ALL"},
                                                            DataSource= 
                                                              {"S3Config": 
                                                                {"Path": item_subset_path,
                                                                 "RoleArn": role_arn}
                                                                },
                                                                Schema= 
                                                                  {"Attributes": 
                                                                    [{"AttributeName": "item_id",
                                                                      "AttributeType": "string",
                                                                      "AttributeCategory": "item_id"}
                                                                    ]
                                                                  },
                                                            EnableVisualization=True)
                                                


In [ ]:
explainability_item_level_arn = create_expainability_response['ExplainabilityArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability(ExplainabilityArn=explainability_item_level_arn))
assert status

In [ ]:
forecast.describe_explainability(ExplainabilityArn=explainability_item_level_arn)

We can also the results on the Amazon Forecast console.

For more details about using the Forecast console to create and view explainabilities, see: https://aws.amazon.com/blogs/machine-learning/understand-drivers-that-influence-your-forecasts-with-explainability-impact-scores-in-amazon-forecast/

![Image View explainability impact scores for specifc items](images/ItemLevelAggregateAllTimepoints.png "Aggregate impact scores for specific items")

From the dropdown, selecting the aggregate impact score across all time-series and time-points in the explainability job shows that Model_Type has an impact score of 0.361, meaning overall Model_Type moderately drives up the forecasted order quantites. 

Customer_Request (the number of customers on the waitlist for an item) has slightly less impact, with a score of 0.2608. 

Loss_Rate (the items damaged during transportation) has an impact of 0.1003, less than half of that of Customer_Request. 

Holidays has almost no impact, with a score of almost 0.

Next, let's selected a specific time-series: Guitar_1

![Image View explainability impact scores for Guitar 1](images/ItemLevelGuitar1AllTimepoints.png "Aggregate impact scores for Guitar 1")

Guitar 1 across the timepoints explained in this job has a very high impact of 1 for Model_Type, meaning this attribute for Guitar 1 for the time-series in this job has a high impact that is increasing the forecasted values. 

Customer_Request has a much lower impact that still increases the forecast values.

Holiday_US has no impact.

Loss_Rate, represented by the bar in red, has an impact of 0.0419, but this impact decreases the forecasted values, driving them lower.

You can also view scores for the items in this job at specific time-points, by selecting a time-point from the drop-down.

## Step 4b. Create Explainability export for specific time-series<a class="anchor" id="itemLevelExplainabilityExport">

Forecast enables you to export a CSV file of Impact scores to an S3 location. These exports are more detailed than the Impact scores displayed in the console.

If you use the “Specific time series” or “Specific time series and time points” scopes, Forecast will also export aggregated impact scores. Exports for the “Specific time series” scope include aggregated normalized scores for the specified time series, and exports for the “Specific time series and time points” scope include aggregated normalized scores for the specified time points.

In [ ]:
explainability_export_name = f"{project}_item_level_explainability_export_{idx}"
explainability_export_destination = f"s3://{bucket_name}/{project}/{explainability_export_name}"

In [ ]:
explainability_export_response = forecast.create_explainability_export(ExplainabilityExportName=explainability_export_name, 
                                                                       ExplainabilityArn=explainability_item_level_arn, 
                                                                       Destination=
                                                                          {"S3Config": 
                                                                            {"Path": explainability_export_destination,
                                                                             "RoleArn": role_arn}
                                                                          })

In [ ]:
explainability_export_item_level_arn = explainability_export_response['ExplainabilityExportArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_level_arn))
assert status

In [ ]:
forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_level_arn)

Now let's load and view the data

In [ ]:
export_data = read_explainability_export(bucket_name, project+"/"+explainability_export_name)

The export for the “Specific time series” scope contains raw and normalized impact scores for the specified time series, as well as a normalized aggregated impact score for all specified time series. The are no raw impact scores for the aggregate because, like with the “Entire forecast” scope, the aggregated scores are already representative of all specified time series.

Impact scores come in two forms: Normalized impact scores and Raw impact scores. Raw impact scores are based on Shapley values and are not scaled or bounded. Normalized impact scores scale the raw scores to a value between -1 and 1 to make comparing scores within the Explainability insight easier.

The export file contains the aggregate impact scores across all time-series in the job across all time-points

In [ ]:
export_data.loc[export_data['item_id'] == "Aggregate"]

The export file also contains the aggregate impact scores across all time-points for each time-series in the job

In [ ]:
export_data.loc[export_data['timestamp'] == "Aggregate"].loc[export_data["item_id"] != "Aggregate"]

From the normalized impact score, we find the for Guitar_1 Model_Type has high impact score close to 1, meaning this attribute is driving up the forecasted values Guitar_1, as the maximum normalized impact score is 1.

Guitar_4 on the other hand has a normalized impact score close to 1 for Customer_Request, meaning this attribut has a higher impact on Guitar_4 than Loss-Rate does.

Guitars 2 and 3 are overall not impacted by these features, with aggregate impact scores of 0. 

### Aggregating impact scores

Forecast imposes a limit of 50 time-series that can be explained per explainabilty job.
If you have more than 50 items to explain, the explainability for all the time-series can be generated in multiple batches. 

From there, if you want to generate an aggregate score for all time-series across explainability jobs, this can be done by taking an average of the noramlized impact scores for each feature. 

We will create one more explainability job, this time with differenent set of items and aggregate the results with those from the first batch. 


In [ ]:
second_item_subset_file = "InstrumentData/second_item_subset.csv"
second_item_subset_df = pd.read_csv(second_item_subset_file, names=['item_id'])
second_item_subset_df.style.hide_index()

Now, save the local item subset to S3

In [ ]:
key = f"{project}/InstrumentData/second_item_subset.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(second_item_subset_file)

In [ ]:
item_subset_path = f"s3://{bucket_name}/{key}"
explainability_name_second_batch = f"{project}_item_level_explainability_2nd_batch_{idx}"

In [ ]:
create_expainability_response=forecast.create_explainability(ExplainabilityName=explainability_name_second_batch, 
                                                            ResourceArn=forecast_arn,
                                                            ExplainabilityConfig={"TimeSeriesGranularity": "SPECIFIC", "TimePointGranularity": "ALL"},
                                                            DataSource= 
                                                              {"S3Config": 
                                                                {"Path": item_subset_path,
                                                                 "RoleArn": role_arn}
                                                                },
                                                                Schema= 
                                                                  {"Attributes": 
                                                                    [{"AttributeName": "item_id",
                                                                      "AttributeType": "string",
                                                                      "AttributeCategory": "item_id"}
                                                                    ]
                                                                  },
                                                            EnableVisualization=True)

In [ ]:
explainability_item_level_batch2_arn = create_expainability_response['ExplainabilityArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability(ExplainabilityArn=explainability_item_level_batch2_arn))
assert status

In [ ]:
forecast.describe_explainability(ExplainabilityArn=explainability_item_level_batch2_arn)

Now export the explainability results for the second batch of items

In [ ]:
explainability_export_name_second_batch = f"{project}_item_level_export_batch2_{idx}"
explainability_export_destination = f"s3://{bucket_name}/{project}/{explainability_export_name_second_batch}"

In [ ]:
explainability_export_response = forecast.create_explainability_export(ExplainabilityExportName=explainability_export_name_second_batch, 
                                                                       ExplainabilityArn=explainability_item_level_batch2_arn, 
                                                                       Destination=
                                                              {"S3Config": 
    {"Path": explainability_export_destination,
     "RoleArn": role_arn}
  })

In [ ]:
explainability_export_item_level_batch2_arn = explainability_export_response['ExplainabilityExportArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_level_batch2_arn))
assert status

In [ ]:
forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_level_batch2_arn)

In [ ]:
export_data_second_batch = read_explainability_export(bucket_name, project+"/"+explainability_export_name)

Concatenate the explainability export results

In [ ]:
export_combined_data = pd.concat([export_data, export_data_second_batch])

Now that we have the results from both explainability jobs, we take an average across over the normalized impact scores for each feature in the data.

In [ ]:
normalized_columns = ['Customer_Request-NormalizedImpactScore', 'Loss_Rate-NormalizedImpactScore','Model_Type-NormalizedImpactScore','Holiday_US-NormalizedImpactScore']
aggregate_impact_scores = pd.DataFrame(export_combined_data[normalized_columns].mean(), columns=['Mean'])
aggregate_impact_scores

Now we have the aggregate noramlized impact scores for all items in both batches.

## 4c. Create Explainability for specific time-series at specific time-points<a class="anchor" id="itemAndTimePointLevelExplainability">

When you specify time points for Forecast Explainability, Amazon Forecast calculates Impact scores for attributes for that specific time range. You can specify up to 500 consecutive time points within the forecast horizon.

The Impact scores can be interpreted as the impact attributes have on a specific time series at a given time.

The modifications to create explainability for specific time-series at specific time-points are:
* In ExplainabilityConfig, set values for TimeSeriesGranularity to “SPECIFIC” and TimePointGranularity to “SPECIFIC”.
```python
ExplainabilityConfig={"TimeSeriesGranularity": "SPECIFIC", "TimePointGranularity": "SPECIFIC"}
```
* Provide a startDateTime and EndDateTime in the request. Impact scores will be generated for all time-points between the startDateTime and  EndDateTime. For example:
```python
EndDateTime="2022-11-30T09:00:00",
StartDateTime="2022-01-30T09:00:00"
```

First, upload the item subset to S3 and set the explainability name

In [ ]:
key = f"{project}/InstrumentData/second_item_subset.csv"
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file(second_item_subset_file)
item_subset_path = f"s3://{bucket_name}/{key}"

explainability_name = f"{project}_item_timepoint_level_explainability_{idx}"

Now create the explainability

In [ ]:
create_expainability_response=forecast.create_explainability(ExplainabilityName=explainability_name, 
                                                             ResourceArn=forecast_arn,
                                                             ExplainabilityConfig={"TimeSeriesGranularity": "SPECIFIC", 
                                                                                   "TimePointGranularity": "SPECIFIC"},
                                                             DataSource= 
                                                              {"S3Config": 
                                                                {"Path": item_subset_path,
                                                                 "RoleArn": role_arn}
                                                              },
                                                             Schema= 
                                                              {"Attributes": 
                                                                [
                                                                  {"AttributeName": "item_id",
                                                                   "AttributeType": "string",
                                                                   "AttributeCategory": "item_id"}
                                                                ]
                                                              },
                                                              EndDateTime="2022-11-30T09:00:00",
                                                              StartDateTime="2022-01-30T09:00:00",
                                                              EnableVisualization=True)
                            

In [ ]:
explainability_item_and_timepoint_level_arn = create_expainability_response['ExplainabilityArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability(ExplainabilityArn=explainability_item_and_timepoint_level_arn))
assert status

In [ ]:
forecast.describe_explainability(ExplainabilityArn=explainability_item_and_timepoint_level_arn)

Now that the explainability job is Active, we can view the results on the Forecast console.

For more details about using the Forecast console to create and view explainabilities, see: https://aws.amazon.com/blogs/machine-learning/understand-drivers-that-influence-your-forecasts-with-explainability-impact-scores-in-amazon-forecast/

From the console, let's look specifically at Guitar_5, by selecting this item from the drop-down. We'll compare the scores for Guitar_5 at two different time-points, to see how the impact scores can change for each forecasted time-point.

![Image View time-series explainability](images/Guitar5TimePoint1Scores.png "Impact scores for Guitar 5 at time-point 1")

The forecasted value for Guitar_5 on Jan 30th 2022 is highly impacted by Customer_Request, which has a normalized impact score of 1. 

Holiday_US has the next highest impact score of 0.3789, followed by Loss_Rate and Model_Type.

Now, let's select the next time-point in the forecast horizon for Guitar_5, on Feb 28th 2022.

![Image View time-series explainability](images/Guitar5TimePoint2Scores.png "Impact scores for Guitar 5 at time-point 2")

For the same Guitar one month later, the Customer_Request impact score changes from 1 to 0.8054.

The impact of Holidays_US drops from 0.3739 in January down to 0 (no impact) in February.

Drilling down to specific time-points can paint a more detailed picture of how each attribute in the data is impacting each item over time.

You still have the option of the viewing the aggregate results for a specific item across time-points or for all items in the explainability job across all time-points by selection 'Aggregate' and 'All' from the drop-down.

## Step 4d. Create Explainability export for specific time-series at specific time-points<a class="anchor" id="itemAndTimePointLevelExplainabilityExport">

In [ ]:
explainability_export_name = f"{project}_item_and_timepoints_level_export_{idx}"
explainability_export_destination = f"s3://{bucket_name}/{project}/{explainability_export_name}"

In [ ]:
explainability_export_response = forecast.create_explainability_export(ExplainabilityExportName=explainability_export_name, 
                                                                       ExplainabilityArn=explainability_item_and_timepoint_level_arn, 
                                                                       Destination=
                                                                          {"S3Config": 
                                                                            {"Path": explainability_export_destination,
                                                                             "RoleArn": role_arn}
                                                                          }
                                                                        )

In [ ]:
explainability_export_item_and_timepoint_level_arn = explainability_export_response['ExplainabilityExportArn']

In [ ]:
status = util.wait(lambda: forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_and_timepoint_level_arn))
assert status

In [ ]:
forecast.describe_explainability_export(ExplainabilityExportArn=explainability_export_item_and_timepoint_level_arn)

In [ ]:
export_data_specific_items_and_time_points = read_explainability_export(bucket_name, project+"/"+explainability_export_name)

The export for the “Specific time series and time points” scope contains raw and normalized impact scores for the specified time series and time points, as well as normalized and raw aggregated impact scores for all specified time points.

We'll take a look at the results for a specific item, Guitar_5

In [ ]:
export_data_specific_items_and_time_points
export_data.loc[export_data['item_id'] == "Guitar_5"]

## Step 5. Cleaning up your Resources<a class="anchor" id="cleanup">

Once we have completed the above steps, we can start to cleanup the resources we created. All delete jobs, except for `delete_dataset_group` are asynchronous, so we have added the helpful `wait_till_delete` function. 
Resource Limits documented <a href="https://docs.aws.amazon.com/forecast/latest/dg/limits.html">here</a>. 

If you want to clean up all the resources generated in this notebook, uncomment the lines in the cells below

Delete explainability exports:

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_explainability_export(ExplainabilityExportArn = explainability_export_model_level_arn)
#util.wait_till_delete(lambda: forecast.delete_explainability_export(ExplainabilityExportArn = explainability_export_item_level_arn)
#util.wait_till_delete(lambda: forecast.delete_explainability_export(ExplainabilityExportArn = explainability_export_item_level_batch2_arn)
#util.wait_till_delete(lambda: forecast.delete_explainability_export(ExplainabilityExportArn = explainability_export_item_and_timepoint_level_arn))

Delete explainabilities:

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_explainability(ExplainabilityArn = explainability_item_level_arn))
#util.wait_till_delete(lambda: forecast.delete_explainability(ExplainabilityArn = explainability_item_level_batch2_arn))
#util.wait_till_delete(lambda: forecast.delete_explainability(ExplainabilityArn = explainability_model_level_arn))
#util.wait_till_delete(lambda: forecast.delete_explainability(ExplainabilityArn = explainability_item_and_timepoint_level_arn))

Delete forecast:

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecast_arn))

Delete predictor:

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictor_arn))

Delete dataset imports for TTS, RTS and IM:

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ts_dataset_import_job_arn))
#util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=rts_dataset_import_job_arn))
#util.wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=im_dataset_import_job_arn))

Delete the datasets for TTS, RTS and IM

In [ ]:
#util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=ts_dataset_arn))
#util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=rts_dataset_arn))
#util.wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=im_dataset_arn))

Delete the dataset group

In [ ]:
#forecast.delete_dataset_group(DatasetGroupArn=dataset_group_arn)

Delete the IAM role

In [ ]:
#util.delete_iam_role( role_name )